In [ ]:
# Import the two main classes FaceAnalyzer and Face
from FaceAnalyzer import FaceAnalyzer, Face
import cv2 as cv

fa = FaceAnalyzer()
# ... Recover an image in RGB format as numpy array (you can use pillow opencv but if you use opencv make sure you change the color space from BGR to RGB)
# Now process the image
image  = cv.imread(f'face.jpg')
fa.process(image)

# Now you can find faces in fa.faces which is a list of instances of object Face
if fa.nb_faces>0:
    print(f"{fa.nb_faces} Faces found")
    # We can get the landmarks in numpy format NX3 where N is the number of the landmarks and 3 is x,y,z coordinates
    print(fa.faces[0].npLandmarks)
    # We can draw all landmarks
    # Get head position and orientation compared to the reference pose (here the first frame will define the orientation 0,0,0)
    pos, ori = fa.faces[0].get_head_posture()


In [2]:
pos

array([[  8.55257914],
       [ -3.49291148],
       [403.49760596]])

In [3]:
ori

array([[-0.09421726],
       [ 0.06091542],
       [-0.01784859]])

In [5]:
Face.get_landmark_pos(fa, index=fa.faces[0].npLandmarks)

c:\Users\Varad\anaconda3\Lib\site-packages\FaceAnalyzer\Face.py:1716: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(region)>=3,"Region should contain at least 3 points")
c:\Users\Varad\anaconda3\Lib\site-packages\FaceAnalyzer\Face.py:1716: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(region)>=3,"Region should contain at least 3 points")
c:\Users\Varad\anaconda3\Lib\site-packages\FaceAnalyzer\Face.py:1716: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(region)>=3,"Region should contain at least 3 points")
c:\Users\Varad\anaconda3\Lib\site-packages\FaceAnalyzer\Face.py:1716: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(region)>=3,"Region should contain at least 3 points")


AttributeError: 'FaceAnalyzer' object has no attribute 'ready'

In [1]:
import cv2
# import mediapipe as mp
from FaceAnalyzer import FaceAnalyzer, Face
# import pyautogui as pyg
# from time import sleep

fa = FaceAnalyzer(1)
cap = cv2.VideoCapture(0)
threshold = 0.4
avgh = 0
avgNum = 3
lastVals = [0 for x in range(avgNum)]
counter = 0

scrollSens = 200

while True:
    success, image = cap.read()
    RGB_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fa.process(RGB_image)

    if(fa.found_faces):
        if(counter >= avgNum):
            counter = 0

        Face.draw_eyes_landmarks(fa.faces[0], image)

        h = Face.get_right_eye_height(fa.faces[0])

        lastVals[counter] = h
        avgh = 0
        for x in lastVals:
            avgh += x
        avgh /= avgNum

        #print([avgh, h, avgh-h])

        if(avgh - h > threshold):
            # pyg.scroll(-scrollSens)
            # print (counter)
            print("BLINK")

    cv2.imshow("Win", image)
    # cv2.waitKey(1)

    if cv2.waitKey(1) == ord('q'):
        exit()

    counter += 1

cap.release()
cv2.destroyAllWindows()

BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK
BLINK


KeyboardInterrupt: 

: 

In [ ]:
# !python.exe -m pip install --upgrade pip
# !pip uninstall mediapipe -y
# !pip install mediapipe
!pip install --upgrade mediapipe

In [1]:
import cv2
import mediapipe as mp
from FaceAnalyzer import FaceAnalyzer, Face
import pyautogui as pyg
import numpy as np
from math import sqrt

fa = FaceAnalyzer(1)
cap = cv2.VideoCapture(0)
threshold = 0.8
avgh = 0
avgNum = 3
lastVals = [0 for x in range(avgNum)]

avgh_l = 0
avgNum_l = 3
lastVals_l = [0 for x in range(avgNum_l)]
counter = 0

scrollSens = 200
y_pred = []
while True:
    success, image = cap.read()
    image = cv2.flip(image,1)
    RGB_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    fa.process(RGB_image)

    if(fa.found_faces):
        Face.draw_eyes_landmarks(fa.faces[0], image)
        # print(fa.faces[0].landmarks)
        # print(Face.get_3d_realigned_landmarks_pos(fa.faces[0], [473]))
        # Face.reset_face_3d_reference_positions(fa.faces[0])
        # Face.draw_landmark_by_index(fa.faces[0], image, 473, thickness=1)
        # position = Face.get_eyes_position(fa.faces[0])
        # print(position)

        real = Face.get_realigned_landmarks_pos(fa.faces[0], [473, 362, 263])
        Face.draw_landmark_by_index(fa.faces[0], image, 473, color=(0,0,255), thickness=2)

        if(counter >= avgNum):
            counter = 0

        # Face.draw_eyes_landmarks(fa.faces[0], image)

        h = Face.get_right_eye_height(fa.faces[0])
        hl = Face.get_left_eye_height(fa.faces[0])

        lastVals[counter] = h
        avgh = 0
        for x in lastVals:
            avgh += x
        avgh /= avgNum
        h = 1 if avgh - h > threshold else 0

        lastVals_l[counter] = hl
        avgh_l = 0

        for x in lastVals:
            avgh_l += x
        avgh_l /= avgNum_l
        hl = 1 if avgh_l - hl > threshold else 0
        #print([avgh, h, avgh-h])
        

        # print(real)

        # h = Face.get_right_eye_height(fa.faces[0])
        # lastVals[counter] = h
        # avgh = 0
        # for x in lastVals:
        #     avgh += x
        # avgh /= avgNum

        landmarks_pos = Face.get_landmarks_pos(fa.faces[0], [473, 362, 263, 475]) #3d array
        # print(landmarks_pos)
        
        '''distn for left (473, 362) || distn for right (473, 263)'''
        pupil = landmarks_pos[0]
        left = landmarks_pos[1]
        right = landmarks_pos[2]

        l_dist = sqrt(((pupil[0]-left[0])**2)+((pupil[1]-left[1])**2)+((pupil[2]-left[2])**2))
        r_dist = sqrt(((pupil[0]-right[0])**2)+((pupil[1]-right[1])**2)+((pupil[2]-right[2])**2))

        # print(l_dist, r_dist, l_dist+r_dist)

        # if(l_dist>22 and r_dist>22): print('center')

        # lastVals[counter] = h
        # avgh = 0
        # for x in lastVals:
        #     avgh += x
        # avgh /= avgNum
        # if(avgh - h > threshold):
        #     print("BLINK")
    counter += 1
    cv2.imshow("Win", image)

    if len(y_pred)==20:
        import statistics
        prediction = statistics.mode(y_pred)
        print(prediction)
        y_pred.clear()

    if(l_dist<24): 
        y_pred.append('left')
        continue
    elif(r_dist<20): 
        y_pred.append('right')
        continue
    else: 
        y_pred.append('center')
    if(h and hl): 
        print('blink')


    if cv2.waitKey(1) == ord('q'):
        break

    

cap.release()
cv2.destroyAllWindows()

c:\Users\Varad\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
center
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
right
left
left
left
left
left
left
left
center
left
center
left
right
left
left
left
left
left
left
left
center
center
blink
center
center
center
left
left
left
left
left
right
right
center
blink
left
center
center
left
center
left
left
left
center
blink
center
center
center
left
left
left
left
center
center
center
left
right
right
center
center
center
center
left
center
left
right
right
center
center
center
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left
left


In [1]:
# 473, 362, 263
left_eye_contour_indices = [
                                    476, # Right
                                    477, # Bottom
                                    474, # Left
                                    475  # Top
                                ]
left_eye_left_right_indices = [
                                    362, # Right
                                    263, # Left
                                ]                                
left_eye_center_index = 473
#################################################################
right_eye_contour_indices = [
                                    471, # right
                                    472, # bottom
                                    469, # left
                                    470  # top
                                ]
right_eye_left_right_indices = [
                                    130, # Right
                                    133, # Left
                                ]                                   
right_eye_center_index = 468    